In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score

In [3]:
# Load Dataframe

df = pd.read_csv('Preprocessed_Data/Final_df.csv')
df.drop(columns=['Unnamed: 0'], inplace=True)
df.head()

,score,thumbsUpCount,accurate,add,ai,also,always,amazing,answer,anything,...,voice,want,way,well,wonderful,work,would,wow,sentiment,text_length
0,5,0,0.0,0.0,0.540835,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.6369,4.0
1,5,0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.4404,2.0
2,5,0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.7096,3.0
3,1,0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.4215,1.0
4,4,0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1372,9.0


# MODELLING

In [4]:
# Split df

X = df.drop(columns=['score'])
y = df['score']

print(f'Shape for X features set: {X.shape}')
print(f'Shape for y target column: {y.shape}')

Shape for X features set: (134618, 103)
Shape for y target column: (134618,)


In [ ]:
# Initialize ML models

models = {
    'RandomForest': RandomForestClassifier(random_state=42, n_jobs=-1),
    'LogisticRegression': LogisticRegression(max_iter=1000, random_state=42, n_jobs=-1),
    'XGBoost': XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='mlogloss', n_jobs=-1)
}